NOMBRES Y APELLIDOS: ALEX HELDER HUANCARA CCOLQQUE \\

CURSO: MINERIA DE DATOS \\

DOCENTE: CARLOS FERNANDO MONTOYA CUBAS

In [2]:
!pip install pyspark

     |████████████████████████████████| 281.3 MB 40 kB/s 
     |████████████████████████████████| 198 kB 52.7 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.0-py2.py3-none-any.whl size=281805912 sha256=f072e59c142463f7f4502c19408273b1c5602b598ff027591982473452b3e180
  Stored in directory: /root/.cache/pip/wheels/0b/de/d2/9be5d59d7331c6c2a7c1b6d1a4f463ce107332b1ecd4e80718
Successfully built pyspark


In [3]:
from pyspark import SparkContext
sc = SparkContext()

In [4]:
import math
from pyspark.sql.functions import *

In [13]:
import pandas as pd

In [7]:
def clean_data(textRDD):
  #Eliminar lineas vacias
  textRDD=textRDD.filter(lambda x : x is not '')
  tupleData = []
  i=1
  for line in textRDD.collect():
    tupla=(i,line)
    tupleData.append(tupla)
    i+=1
  return tupleData

""" Creamos la bolsa de palabras"""
def bow(data, particiones = 4):
  #obtenemos la frecuencia de cada palabra por documento
  #Creamos el RDD
  linesRDD=sc.parallelize(data,particiones)
  parTokenRDD=linesRDD.flatMap(lambda x: [((x[0],i.lower(),len(x[1].split())),1) for i in x[1].split()])
  #filtramos palabras de tamaño mayor a 2 letras
  parTokenRDD2=parTokenRDD.filter(lambda x: len(x[0][1])>2)
  #obtenemos la frecuencia de cada palabra por documento
  frecPerDocumentRDD=parTokenRDD2.reduceByKey(lambda x,y:x+y)
  return frecPerDocumentRDD

"""Hallando el TF"""
def TF(datos):
  bowRDD=bow(datos)
  # Mapear el RDD para obetener el token
  tfRDD=bowRDD.map(lambda x: (x[0][1],(x[0][0],x[1]/x[0][2])))
  return tfRDD

"""Hallando el IDF"""
def IDF(datos):
  # hallamos la cantidad de documentos
  cantidadDoc=len(datos)
  # hallamos la bolsa de palabras
  bowRDD=bow(datos)
  # Mapeamos para colocar cada token como valor clave.
  tokenRDD=bowRDD.map(lambda x: (x[0][1],(x[0][0],x[1],1)))
  # Extraemos el token y su ocurrencia por documento
  tokenOcurrenciaDocRDD=tokenRDD.map(lambda x:(x[0],x[1][2]))
  # contamos la contidad de documentos que contienen un token
  docPerTokenRDD=tokenOcurrenciaDocRDD.reduceByKey(lambda x,y:x+y)
  #calculamos el logaritmo para el IDF
  idfRDD=docPerTokenRDD.map(lambda x: (x[0],math.log10(cantidadDoc/x[1])))
  return idfRDD

"""juntamos el TF de las frecuencias y los valores de IDF"""
def TF_IDF(datos):
  # valores tf
  tfRDD = TF(datos)
  # puntuaciones idf
  idfRDD = IDF(datos)
  # unimos los valores de TF e IDF
  tfIdfRDD = tfRDD.join(idfRDD)
  # tf * idf (multiplicamos los valores)
  tfIdfRDD = tfIdfRDD.map(lambda x: (x[1][0][0],(x[0],x[1][0][1],x[1][1],x[1][0][1]*x[1][1]))).sortByKey()
  # organizacion para mostrado de datos
  tfIdfRDD = tfIdfRDD.map(lambda x: (x[0],x[1][0],x[1][1],x[1][2],x[1][3]))
  return tfIdfRDD

In [12]:
# Prueba
twittsRDD = sc.textFile('twitts.txt',4)
twittsRDD = clean_data(twittsRDD)
tf_idf = TF_IDF(twittsRDD)

# Mostrar los valores de tf-idf
tf_idf.collect()

[(1, 'store', 0.1, 1.105595616949241, 0.11055956169492409),
 (1, 'diein', 0.1, 1.399423217328898, 0.1399423217328898),
 (1, 'retweet', 0.1, 1.399423217328898, 0.1399423217328898),
 (1, 'agree', 0.1, 1.3690792008697128, 0.13690792008697128),
 (1, 'stage', 0.1, 1.3798696809532192, 0.13798696809532193),
 (1, 'nyc', 0.1, 1.353380100659997, 0.13533801006599971),
 (1, 'protester', 0.1, 1.388142206919209, 0.13881422069192093),
 (1, 'protest', 0.1, 1.377146822617746, 0.1377146822617746),
 (1, 'anger', 0.1, 1.388142206919209, 0.13881422069192093),
 (1, 'misplace', 0.1, 1.390935107103379, 0.1390935107103379),
 (2, 'range', 0.08333333333333333, 2.2195389692560776, 0.18496158077133978),
 (2, 'trading', 0.08333333333333333, 1.928054291498327, 0.1606711909581939),
 (2, 'stock', 0.08333333333333333, 1.457415164306585, 0.1214512636922154),
 (2, 'mkt', 0.08333333333333333, 1.968482948553935, 0.1640402457128279),
 (2, 'update', 0.08333333333333333, 1.3937460844372076, 0.11614550703643396),
 (2, 'apple',

La primera columna es la identificacion del documento, la segunda columna las palabras o tokens, la tercera columna el TF, la cuarta columna el IDF, y la ultima columna es el TF-IDF

In [14]:
# mostrar dataFrame
df = tf_idf.collect()
pd.DataFrame(df,columns=['Id Documento','Token','TF','IDF', 'TF-IDF'])

,Id Documento,Token,TF,IDF,TF-IDF
0,1,store,0.100000,1.105596,0.110560
1,1,diein,0.100000,1.399423,0.139942
2,1,retweet,0.100000,1.399423,0.139942
3,1,agree,0.100000,1.369079,0.136908
4,1,stage,0.100000,1.379870,0.137987
...,...,...,...,...,...
26635,3812,computer,0.142857,1.034724,0.147818
26636,3812,future,0.142857,1.061439,0.151634
26637,3813,work,0.333333,1.581267,0.527089
26638,3813,great,0.333333,1.865263,0.621754
